In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torch.utils.data as dt
import torchvision.transforms as transforms
import time
import os
import random
from PIL import Image
from dataset import AutoDataset
from networks import AutoEncoder
%matplotlib widget

In [2]:
batch = 64
epoch = 30
lr = 0.0001

In [3]:
#function converts one folder of pictures into
def autoencoder_dataset(path, set_percent = 0.1):
    transform = transforms.Compose([transforms.Grayscale(), transforms.ToTensor(), transforms.Normalize((0.5), (0.5/3))])   #Make sure images are tensors, lie within 0 and 1
    dataset = AutoDataset(path, transform = transform) #need to implement   #Turn the images into a dataset
    set_size = int( set_percent * len( dataset ) )                                                        #Length of test and validation sets
    return dt.random_split(dataset, [len( dataset ) - 2 * set_size, set_size, set_size])                #Return datasets in order of training set, validation set, test setde

In [4]:
train, val, test = autoencoder_dataset('Data\Hilbert\Autoencoder')

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
train_loader = dt.DataLoader(train, batch_size = 64)
val_loader = dt.DataLoader(val, batch_size = 64)
test_loader = dt.DataLoader(test, batch_size = 64)

In [7]:
def get_accuracy(model, loader, criterion, batch_size):
    """ Compute the accuracy of the `model` across a dataset `data`
    
    Example usage:
    
    >>> model = MyRNN() # to be defined
    >>> get_accuracy(model, valid) # the variable `valid` is from above
    """
    model = model.cuda()
    total_loss = 0.0
    total_err = 0.0
    total_epoch = 0
    for i, picture in enumerate(loader, 0):
        picture = picture.cuda()
        outputs = model(picture)
        loss = criterion(outputs, picture)
        corr = torch.sum(torch.eq(picture, outputs))
        #total_err += int(corr.sum())
        total_loss += loss.item()
        total_epoch += len(picture)
    err = float(total_err) / total_epoch
    loss = float(total_loss) / (i + 1) /batch_size
    return err, loss

In [8]:
def pathy(epoch, batch_size, lr):
    model_path = os.path.join('Data','Training', f'epoch_{epoch}_batch{batch_size}_lr{lr}')
    return model_path

In [9]:
def train_auto(net, batch_size = 64, lr = 0.01, epoch = 30):
    net = net.cuda()
    random.seed(1000)
    np.random.seed(1000)
    torch.manual_seed(1000)
    torch.cuda.manual_seed(1000)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=lr)
    
    train_err = np.zeros(epoch)
    train_loss = np.zeros(epoch)
    val_err = np.zeros(epoch)
    val_loss = np.zeros(epoch)
    
    start_time = time.time()
    
    for epoch in range(epoch):  # loop over the dataset multiple times
        total_train_loss = 0.0
        total_train_err = 0.0
        total_epoch = 0
        for i, picture in enumerate(train_loader):
            
            picture = picture.cuda()
            #print(picture.shape)
            
            optimizer.zero_grad()
            outputs = net(picture)
            loss = criterion(outputs, picture)
    
            loss.backward()
            optimizer.step()

            corr = torch.sum(torch.eq(picture, outputs))
            total_train_err += int(corr.sum())
            total_train_loss += loss.item()
            total_epoch += len(picture)
        
        train_err[epoch] = float(total_train_err) / total_epoch
        train_loss[epoch] = float(total_train_loss) /  (i+1) / batch_size
        val_err[epoch], val_loss[epoch] = get_accuracy(net, val_loader, criterion, batch_size)
        print(("Epoch {}: Train loss: {} | "+
                "Validation loss: {}").format(
                   epoch + 1,
                   train_loss[epoch],
                   val_loss[epoch]))
        # Save the current model (checkpoint) to a file
        model_path = pathy(epoch, batch_size, lr)
        torch.save(net.state_dict(), model_path)
    print('Finished Training')
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))
    # Write the train/test loss/err into CSV file for plotting later
    #print(net.state_dict())
    
    np.savetxt("{}_train_loss.csv".format(model_path), train_loss)
    np.savetxt("{}_val_loss.csv".format(model_path), val_loss)

    return criterion

In [10]:
def plot_training_curve(path):
    """ Plots the training curve for a model run, given the csv files
    containing the train/validation error/loss.

    Args:
        path: The base path of the csv files produced during training
    """
    import matplotlib.pyplot as plt
    train_loss = np.loadtxt("{}_train_loss.csv".format(path))
    val_loss = np.loadtxt("{}_val_loss.csv".format(path))
    plt.title("Train vs Validation Error")
    n = len(train_loss) # number of epochs
    plt.title("Train vs Validation Loss")
    plt.plot(range(1,n+1), train_loss, label="Train")
    plt.plot(range(1,n+1), val_loss, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(loc='best')
    plt.show()

In [11]:
auto = AutoEncoder()
crit = train_auto(auto, batch_size=batch, lr = lr, epoch = epoch)

Epoch 1: Train loss: 0.01392189982379878 | Validation loss: 0.008104333389214995
Epoch 2: Train loss: 0.00706560218209209 | Validation loss: 0.0059569750060614254
Epoch 3: Train loss: 0.005448610962582957 | Validation loss: 0.0051044769324715385
Epoch 4: Train loss: 0.004805937300257248 | Validation loss: 0.004606304021543378
Epoch 5: Train loss: 0.004449227181718126 | Validation loss: 0.004329169735144736
Epoch 6: Train loss: 0.004209750569339649 | Validation loss: 0.004115877392750357
Epoch 7: Train loss: 0.004003401618264726 | Validation loss: 0.003917428216857226
Epoch 8: Train loss: 0.003825133114444853 | Validation loss: 0.0037520815424126755
Epoch 9: Train loss: 0.003672015057779197 | Validation loss: 0.0036089447441718307
Epoch 10: Train loss: 0.003547802323016744 | Validation loss: 0.0035051188687683565
Epoch 11: Train loss: 0.003450681910568107 | Validation loss: 0.003413074020896619
Epoch 12: Train loss: 0.003372480495013452 | Validation loss: 0.0033416268956445827
Epoch 13:

In [14]:
plot_training_curve(pathy(epoch=epoch - 1, batch_size=batch, lr = lr))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
torch.save(auto.state_dict(), os.path.join('Model', f'epoch{epoch}_batch{batch}_lr{lr}.pth'))